### Tanks 4.09d Vertical Fixed Roof Tank Calculations 

In [2]:
import math
import pandas as pd
import numpy as np

In [3]:
class VerticalFixedRoofTank:
    def __init__(self, tkshellht, skliqht, tkrfslope, diameter,
                 ins, solarabs, tax, tan):
        
        '''
        tax: Average Daily Maximum Ambient Temperature, user defined, in deg F.
        tan: Average Daily Minimum Ambient Temperature, user defined, in deg F.
        solarabs: Tank surface solar absorptance, user defined, depends on tank color.
        ins: Average daily total insulation, defined to be 1,491 (btu/ft^2*d) (from Table 7.1-7)
        '''
        self.tkshellht = tkshellht
        self.skliqht = skliqht
        self.tkrfslope = tkrfslope
        self.diameter = diameter
        self.ins = ins
        self.solarabs = solarabs
        
        #Values are in deg. F
        self.tax = tax
        self.tan = tan
        
        self.rad_ = (1/2.)*self.diameter
        self.hro_ = (1/3.)*self.rad_*self.tkrfslope
        
        self.hvo = self.tkshellht - self.skliqht + self.hro_
        
        #Returns values in def. Rankine
        self.tax_r = self.tax + 459.7
        self.tan_r = self.tan + 459.7

        self.taa = (self.tax_r + self.tan_r) * (1/2.)
        self.tb = self.taa + (0.003*self.solarabs*self.ins)
        
    def vv(self):
        '''
        Returns the vapor space volume of the storage tank.
        '''
        return (math.pi)*(1/4.)*(self.diameter**2)*self.hvo
    
    def tla(self):
        '''
        Returns the Daily Average Liquid Surface Temperature, as caluclated from Eqn: 1-28
        '''

        return (0.4*self.taa) + (0.6*self.tb) + ((0.005*self.solarabs)*(self.ins))
    
    def tv(self):
        '''
        Returns average vapor temperrature, in Rankine as caluclated from Eqn: 1-33
        '''
        return (0.7*self.taa) + (0.3*self.tb) + (0.009*self.solarabs*self.ins)

In [4]:
tank = VerticalFixedRoofTank(tkshellht=12, skliqht=8, tkrfslope=0.0625, diameter=6,
                         ins=1491, solarabs=0.25, tax=63.5, tan=37.9)

In [5]:
tank.vv()

114.86448139687681

In [6]:
tank.tla()

512.9346999999999

In [7]:
tank.tv()

514.0902249999999

In [8]:
cyclohexane_A = 6.845
cyclohexane_B = 1203.5
cyclohexane_C = 222.86

toluene_A = 7.017
toluene_B = 1377.6
toluene_C = 222.64

benzene_A = 6.906
benzene_B = 1211.0
benzene_C = 220.79

tla_c = (tank.tla() - 491.7)/1.8

In [9]:
def calVaporPressure(an_a, an_b, an_c, tla_c):
    '''
    Returns a list of the calculated vapor pressures based on the Antone coefficients. 
    Element 0 of the list of the value in PSIA and Element 1 of the list is the value in mmHg.
    tla_c: Daily Average Liquid Surface Temperature in deg C.
    '''
    return [10**(an_a - ((an_b) / (tla_c + an_c)))/ 51.715, 10**(an_a - ((an_b) / (tla_c + an_c)))]

In [10]:
phMe_vp = calVaporPressure(an_a=toluene_A, an_b=toluene_B, an_c=toluene_C, tla_c=tla_c)
phH_vp = calVaporPressure(an_a=benzene_A, an_b=benzene_B, an_c=benzene_C, tla_c=tla_c)
cyH_vp = calVaporPressure(an_a=cyclohexane_A, an_b=cyclohexane_B, an_c=cyclohexane_C, tla_c=tla_c)

In [11]:
# TODO: Calculate mole fractions of mixture and elementwise partial pressures.

In [12]:
comp_amt = [2812.0, 258, 101]
comp_mw = [78.11, 92.14, 84.16]
comp_name = ['Benzene', 'Toluene', 'Cyclohexane']
comp_vp = [phH_vp[0], phMe_vp[0], cyH_vp[0]]

df = pd.DataFrame({'component': comp_name, 'amount_lb': comp_amt, 'mw_lbs_mol': comp_mw, 'comp_vp': comp_vp})
df['comp_mole'] = df['amount_lb'] / df['mw_lbs_mol']
tot_moles = np.sum(df['comp_mole'].tolist())

df['comp_mole_xi'] = df['comp_mole'] / tot_moles
df['comp_partial'] = df['comp_mole_xi'] * df['comp_vp']
vp_mixture = np.sum(df['comp_partial'].tolist())

df['comp_vap_mole_frac'] = df['comp_partial'] / vp_mixture
df['comp_vapor_mw_xi'] = df['mw_lbs_mol'] * df['comp_vap_mole_frac']
vapor_mw = np.sum(df['comp_vapor_mw_xi'].tolist())

In [13]:
df

,component,amount_lb,mw_lbs_mol,comp_vp,comp_mole,comp_mole_xi,comp_partial,comp_vap_mole_frac,comp_vapor_mw_xi
0,Benzene,2812.0,78.11,0.967678,36.000512,0.899997,0.870907,0.946834,73.957207
1,Toluene,258.0,92.14,0.267411,2.800087,0.070001,0.018719,0.020351,1.875138
2,Cyclohexane,101.0,84.16,1.006056,1.200095,0.030002,0.030184,0.032815,2.761709


In [14]:
vapor_mw

78.59405480262626

In [17]:
vp_mixture

0.9198100479313434

In [15]:
#TODO: Calculate Stock Density (Wv) using Eqn: 1-22

In [19]:
def stockDensity(mv, pva, tv):
    '''
    Uses R = 10.731 psia*ft3 / lb-mole* deg R
    Returns stock density in units of lbs/ft3
    '''
    return ((mv*pva) / (10.731 *tv))

In [20]:
stockDensity(mv=vapor_mw, pva=vp_mixture, tv=tank.tv())

0.013104133345548597